In [56]:
import json 
import os 
import requests
from dataclasses import dataclass
from typing import Dict, List
from neo4j import GraphDatabase
from utils.utilities import *
from utils.neo4j_conn import *
from utils.neo4j_schema import *
from utils.graph_utils import *

@dataclass
class Config:
    neo4j_uri: str = "neo4j+s://4017a08b.databases.neo4j.io"
    neo4j_user: str = "neo4j"
    neo4j_password: str = "NMa5reBMrNMfIJvMFLKy6lsnKqYQR_q4EajPYzH8pQk"
    groq_api_key: str = "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"
    database: str = "neo4j"
    
    @property
    def groq_headers(self) -> Dict:
        return {
            "Authorization": f"Bearer {self.groq_api_key}",
            "Content-Type": "application/json"
        }

In [57]:
f = json.load(open('evaluation_query.json'))
cnt = -1
cypher_queries = []
for i in f:
    # print(i)
    cnt += 1
    if (cnt % 3 == 0):
        print(i["question"])
        cypher_queries.append(i["question"])

Here are the 20 generated questions, along with their reasoning and Cypher statements:
Cypher: `MATCH (c:Case {primaryid: 10})-[:IS_PRIMARY_SUSPECT]->(d:Drug) RETURN d.name`
Cypher: `MATCH (c:Case {primaryid: 107526832})-[:FALLS_UNDER]->(ag:AgeGroup) RETURN ag.ageGroup`
Cypher: `MATCH (c:Case {primaryid: 109543792}) RETURN c.reporterOccupation`
Cypher: `MATCH (c:Case {primaryid: 172775763})-[:RESULTED_IN]->(o:Outcome) RETURN o.outcome`
Cypher: `MATCH (m:Manufacturer)-[:REGISTERED]->(c:Case {primaryid: 135402785}) RETURN m.manufacturerName`
Cypher: `MATCH (c:Case {primaryid: 172775763})-[:IS_PRIMARY_SUSPECT]->(d:Drug {name: 'DEXAMETHASONE'}) RETURN d.doseAmount`
Cypher: `MATCH (d:Drug {id: 7160})-[:IS_SECONDARY_SUSPECT]->(c:Case) RETURN c.primaryid`
Cypher: `MATCH (c:Case {primaryid: 1376832112})-[:IS_SECONDARY_SUSPECT]->(d:Drug {name: 'AFINITOR'}) RETURN d.route`
Cypher: `MATCH (c:Case {primaryid: 172652664}) RETURN c.age`
Cypher: `MATCH (c:Case {primaryid: 172652664})-[:HAS_REACTION]-

In [58]:
refined_queries = []

def preprocess(query):
    refined_query = ""
    l = query.split("`")
    print(l)
    refined_queries.append(l[1])
    

# print(cypher_queries[1:])

for i in cypher_queries[1:]:
    preprocess(i)
#     preprocess(i["question"])

print(refined_queries)

['Cypher: ', 'MATCH (c:Case {primaryid: 10})-[:IS_PRIMARY_SUSPECT]->(d:Drug) RETURN d.name', '']
['Cypher: ', 'MATCH (c:Case {primaryid: 107526832})-[:FALLS_UNDER]->(ag:AgeGroup) RETURN ag.ageGroup', '']
['Cypher: ', 'MATCH (c:Case {primaryid: 109543792}) RETURN c.reporterOccupation', '']
['Cypher: ', 'MATCH (c:Case {primaryid: 172775763})-[:RESULTED_IN]->(o:Outcome) RETURN o.outcome', '']
['Cypher: ', 'MATCH (m:Manufacturer)-[:REGISTERED]->(c:Case {primaryid: 135402785}) RETURN m.manufacturerName', '']
['Cypher: ', "MATCH (c:Case {primaryid: 172775763})-[:IS_PRIMARY_SUSPECT]->(d:Drug {name: 'DEXAMETHASONE'}) RETURN d.doseAmount", '']
['Cypher: ', 'MATCH (d:Drug {id: 7160})-[:IS_SECONDARY_SUSPECT]->(c:Case) RETURN c.primaryid', '']
['Cypher: ', "MATCH (c:Case {primaryid: 1376832112})-[:IS_SECONDARY_SUSPECT]->(d:Drug {name: 'AFINITOR'}) RETURN d.route", '']
['Cypher: ', 'MATCH (c:Case {primaryid: 172652664}) RETURN c.age', '']
['Cypher: ', 'MATCH (c:Case {primaryid: 172652664})-[:HAS_RE

In [59]:

class Neo4jConnector:
    def __init__(self, config: Config):
        self.driver = GraphDatabase.driver(
            config.neo4j_uri,
            auth=(config.neo4j_user, config.neo4j_password)
        )
        self.database = config.database
        
    def close(self):
        self.driver.close()
        
    def __enter__(self):
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
        
    def fetch_schema(self) -> Dict:
        """Fetch and return the database schema"""
        with self.driver.session(database=self.database) as session:
            result = session.run("CALL db.schema.visualization()")
            record = result.single()
            print("========================= SCHEMA GEN ========================")
            print(record)
            return {
                "nodes": record["nodes"],
                "relationships": record["relationships"]
            }
            
    def fetch_sample_values(self, limit: int = 5) -> List:
        """Fetch sample values from the database"""
        with self.driver.session(database=self.database) as session:
            query = """
            MATCH (n)
            WHERE rand() > 0.6
            RETURN n
            LIMIT $limit
            """
            result = session.run(query, limit=limit)
            return [record["n"] for record in result]

In [60]:
URI = "neo4j+s://4017a08b.databases.neo4j.io"
PWD = "NMa5reBMrNMfIJvMFLKy6lsnKqYQR_q4EajPYzH8pQk"
# Initialize the Neo4j connector module
graph = Neo4jGraph(url=URI, username='neo4j', password=PWD, database='neo4j')

# Module to extract data from the graph
gutils = Neo4jSchema(url=URI, username='neo4j', password=PWD, database='neo4j')

In [61]:
q = "MATCH (n:AgeGroup) RETURN n LIMIT 2"
print(gutils.run_cypher(q))

[{'n': {'id': 142, 'ageGroup': 'Adult'}}, {'n': {'id': 143, 'ageGroup': 'Elderly'}}]


In [62]:
for query in refined_queries: 
    print("Query: ", query)
    print("Results: ", gutils.run_cypher(query))

Query:  MATCH (c:Case {primaryid: 10})-[:IS_PRIMARY_SUSPECT]->(d:Drug) RETURN d.name
Results:  []
Query:  MATCH (c:Case {primaryid: 107526832})-[:FALLS_UNDER]->(ag:AgeGroup) RETURN ag.ageGroup
Results:  [{'ag.ageGroup': 'Elderly'}]
Query:  MATCH (c:Case {primaryid: 109543792}) RETURN c.reporterOccupation
Results:  [{'c.reporterOccupation': 'Hospital'}]
Query:  MATCH (c:Case {primaryid: 172775763})-[:RESULTED_IN]->(o:Outcome) RETURN o.outcome
Results:  [{'o.outcome': 'Hospitalization - Initial or Prolonged'}, {'o.outcome': 'Other Serious (Important Medical Event)'}, {'o.outcome': 'Life-Threatening'}]
Query:  MATCH (m:Manufacturer)-[:REGISTERED]->(c:Case {primaryid: 135402785}) RETURN m.manufacturerName
Results:  [{'m.manufacturerName': 'NOVARTIS'}]
Query:  MATCH (c:Case {primaryid: 172775763})-[:IS_PRIMARY_SUSPECT]->(d:Drug {name: 'DEXAMETHASONE'}) RETURN d.doseAmount
Results:  [{'d.doseAmount': None}]
Query:  MATCH (d:Drug {id: 7160})-[:IS_SECONDARY_SUSPECT]->(c:Case) RETURN c.primaryi